In [1]:
!pip install -q kaggle
!cp kaggle.json ~/.kaggle/
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d balraj98/massachusetts-roads-dataset

cp: cannot stat 'kaggle.json': No such file or directory


Saving kaggle.json to kaggle.json
100% 5.75G/5.76G [00:46<00:00, 140MB/s]
100% 5.76G/5.76G [00:46<00:00, 132MB/s]


In [2]:
!unzip massachusetts-roads-dataset.zip
!rm -rf massachusetts-roads-dataset.zip

Archive:  massachusetts-roads-dataset.zip
  inflating: label_class_dict.csv    
  inflating: metadata.csv            
  inflating: tiff/test/10378780_15.tiff  
  inflating: tiff/test/10828720_15.tiff  
  inflating: tiff/test/11128870_15.tiff  
  inflating: tiff/test/11278840_15.tiff  
  inflating: tiff/test/11728825_15.tiff  
  inflating: tiff/test/12328750_15.tiff  
  inflating: tiff/test/15928855_15.tiff  
  inflating: tiff/test/16078870_15.tiff  
  inflating: tiff/test/17878735_15.tiff  
  inflating: tiff/test/17878780_15.tiff  
  inflating: tiff/test/17878885_15.tiff  
  inflating: tiff/test/18028750_15.tiff  
  inflating: tiff/test/18178780_15.tiff  
  inflating: tiff/test/18328735_15.tiff  
  inflating: tiff/test/18328780_15.tiff  
  inflating: tiff/test/18328960_15.tiff  
  inflating: tiff/test/18478735_15.tiff  
  inflating: tiff/test/18478900_15.tiff  
  inflating: tiff/test/18478930_15.tiff  
  inflating: tiff/test/20278885_15.tiff  
  inflating: tiff/test/20728960_15.tiff  


In [7]:
import torch
import torch.nn as nn
import torchvision.models as models

# Создаем экземпляр модели
model = models.mobilenet_v2()#.cuda
# model

In [8]:
# model.classifier = nn.Sequential(
#     nn.Conv2d(1280, num_classes, kernel_size=1),
#     nn.ReLU(inplace=True),
#     nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
# )

In [10]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().cuda()

In [11]:
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
trans = transforms.Compose([transforms.Resize((256,256)),
                            transforms.ToTensor()])

In [12]:
train_data_path = "./tiff/train"
train_labels_path = "./tiff/train_labels"
val_data_path = "./tiff/val"
val_labels_path = "./tiff/val_labels"

In [20]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
class Roads_Data(torch.utils.data.Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform
        self.images = os.listdir(image_folder)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.images[idx])
        mask_path = os.path.join(self.mask_folder, self.images[idx].replace(".tif", "_mask.tif"))

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

train_data = Roads_Data(image_folder=train_data_path,
                           mask_folder=train_labels_path,
                           transform=transform)

val_data = Roads_Data(image_folder=val_data_path,
                      mask_folder=val_labels_path,
                      transform=transform)


train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)



In [21]:
for epoch in range(3):
  for images, masks in train_loader:
    images = images.cuda()
    masks = masks.cuda()
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, masks)
    loss.backward()
    optimizer.step()

  model.eval()
  with torch.no_grad():
    for images, masks in val_loader:
      images = images.cuda()
      masks = masks.cuda()
      outputs = model(images)
      loss = criterion(outputs, masks)
  print(f'Epoch: {epoch+1}/3')

FileNotFoundError: [Errno 2] No such file or directory: './tiff/train_labels/26429275_15_mask.tiff'